In [ ]:
# @title Install required packages (if needed)
!pip install --upgrade --quiet google-cloud-storage gitpython

In [ ]:
# @title Set project variables

PROJECT = !gcloud config get-value project
PROJECT_ID = PROJECT[0]
# define project information manually if the above code didn't work
if PROJECT_ID == "(unset)":
  PROJECT_ID = "[your-project-id]" # @param {type:"string"}

print(f"Project ID: {PROJECT_ID}")

LOCATION = "us-central1" # @param {type:"string"}
REPO_URL = "https://github.com/roitraining/flipped-class-genai-sec" # @param {type:"string"}

In [ ]:
# @title Clone Github Repo and Copy files to Cloud Storage
import os
import uuid
from google.cloud import storage
from git import Repo

# Step 1: Clone the GitHub repo (if not already cloned)
repo_url = REPO_URL
repo_folder = "flipped-class-genai-sec"

if not os.path.exists(repo_folder):
    Repo.clone_from(repo_url, repo_folder)
    print("Repo cloned.")
else:
    print("Repo already exists.")

# Step 2: Create a unique bucket name
import random
import string

def generate_suffix(length=6):
    return ''.join(random.choices(string.ascii_lowercase + string.digits, k=length))

bucket_suffix = generate_suffix()
bucket_name = f"medical-records-{bucket_suffix}"
print(f"Creating bucket: {bucket_name}")


# Step 3: Create the bucket
storage_client = storage.Client(project=PROJECT_ID)
bucket = storage_client.bucket(bucket_name)
bucket.storage_class = "STANDARD"

new_bucket = storage_client.create_bucket(bucket, location=LOCATION)
print(f"Bucket {bucket_name} created in {LOCATION}.")

# Step 4: Upload files from synthetic-medical-records folder to bucket
local_folder = os.path.join(repo_folder, "synthetic-medical-records")

for filename in os.listdir(local_folder):
    file_path = os.path.join(local_folder, filename)

    if os.path.isfile(file_path):
        blob = new_bucket.blob(filename)
        blob.upload_from_filename(file_path)
        print(f"Uploaded {filename}")

print("All files uploaded successfully.")
